In [1]:
'''basic packages'''
import os
import pandas as pd
import numpy as np
from sklearn import datasets, tree, linear_model
import sklearn.ensemble as ek
import seaborn as sns
import queue
import cv2          #影像處理
import scipy.misc   #影像處理
from tqdm import tqdm_notebook as tqdm #進度條
import matplotlib.pyplot as plt
plt.rcParams['font.family']='SimHei' #顯示中文
import matplotlib.lines as mlines
%matplotlib inline
import warnings
# warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)

from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [2]:
# Load in the train datasets
train = pd.read_csv('/data/examples/may_the_4_be_with_u/show_me_the_money/train.csv',
#                     names=['id','step','type','amount','nameOrig','oldbalanceOrg','newbalanceOrig','nameDest','oldbalanceDest','newbalanceDest','Fraud'],
                    encoding = "utf-8")
# train = train.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
#                         'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest', 'Fraud':'isFraud'})

test = pd.read_csv('/data/examples/may_the_4_be_with_u/show_me_the_money/test.csv',
#                    names=['id','step','type','amount','nameOrig','oldbalanceOrg','newbalanceOrig','nameDest','oldbalanceDest','newbalanceDest'],
                   encoding = "utf-8")
# test = test.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
#                         'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest', 'Fraud':'isFraud'})

In [3]:
train.isnull().values.any()

False

In [4]:
train.head()

,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,Fraud
0,0,323,PAYMENT,9092.69,C749189758,0.0,0.00,M1939669337,0.00,0.00,0.0
1,1,164,CASH_IN,23468.75,C525009948,0.0,23468.75,C65353523,77300.17,53831.42,0.0
2,2,228,CASH_OUT,39688.11,C1311063915,0.0,0.00,C1581032686,168913.26,208601.37,0.0
3,3,229,CASH_OUT,179016.96,C700594269,103485.0,0.00,C90149178,2483034.47,2662051.42,0.0
4,4,709,CASH_OUT,77155.99,C646560157,143307.0,66151.01,C671337546,21213865.83,21291021.82,0.0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3817572 entries, 0 to 3817571
Data columns (total 11 columns):
id                int64
step              int64
type              object
amount            float64
nameOrig          object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest          object
oldbalanceDest    float64
newbalanceDest    float64
Fraud             float64
dtypes: float64(6), int64(2), object(3)
memory usage: 320.4+ MB


In [6]:
test.shape

(2545047, 10)

In [7]:
test.isnull().values.any()

False

In [8]:
test.head()

,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest
0,1,329,PAYMENT,41909.68,C1965438439,18391.00,0.00,M296912999,0.00,0.00
1,2,205,CASH_IN,57645.11,C2061644565,3162376.19,3220021.31,C2046839454,701205.57,643617.16
2,3,299,PAYMENT,41911.35,C630506216,9218.00,0.00,M1895361660,0.00,0.00
3,4,165,PAYMENT,10297.71,C1059151844,29739.00,19441.29,M830221248,0.00,0.00
4,5,346,TRANSFER,143577.58,C1707397780,115.00,0.00,C1234376661,1184548.85,1328126.42


### 匯出
train.to_csv('show_me_the_money_train.csv',index=False)

test.to_csv('show_me_the_money_test.csv',index=False)

In [9]:
X = train.drop(['id', 'type', 'nameOrig', 'nameDest'], axis = 1)

In [10]:
X_test = test.drop(['id', 'type', 'nameOrig', 'nameDest'], axis = 1)

In [11]:
X_test.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,329,41909.68,18391.00,0.00,0.00,0.00
1,205,57645.11,3162376.19,3220021.31,701205.57,643617.16
2,299,41911.35,9218.00,0.00,0.00,0.00
3,165,10297.71,29739.00,19441.29,0.00,0.00
4,346,143577.58,115.00,0.00,1184548.85,1328126.42


[^1]: [Predicting Fraud in Financial Payment Services | Kaggle](https://www.kaggle.com/arjunjoshua/predicting-fraud-in-financial-payment-services)

In [12]:
X.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,Fraud
0,323,9092.69,0.0,0.00,0.00,0.00,0.0
1,164,23468.75,0.0,23468.75,77300.17,53831.42,0.0
2,228,39688.11,0.0,0.00,168913.26,208601.37,0.0
3,229,179016.96,103485.0,0.00,2483034.47,2662051.42,0.0
4,709,77155.99,143307.0,66151.01,21213865.83,21291021.82,0.0


In [13]:
X_test.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,329,41909.68,18391.00,0.00,0.00,0.00
1,205,57645.11,3162376.19,3220021.31,701205.57,643617.16
2,299,41911.35,9218.00,0.00,0.00,0.00
3,165,10297.71,29739.00,19441.29,0.00,0.00
4,346,143577.58,115.00,0.00,1184548.85,1328126.42


In [14]:
Y = X['Fraud']
X = X.drop(['Fraud'], axis=1)
Y.shape

(3817572,)

In [ ]:
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz

In [ ]:
randomState = 5
np.random.seed(randomState)

X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size = 0.2, \
                                                random_state = randomState)

In [ ]:
import xgboost

eval_set =  [(X_valid, y_valid)]
# parameters={'learning_rate':[0.05,0.1,0.2]}
model = xgboost.XGBClassifier()
# best_model = sklearn.model_selection.GridSearchCV(model, parameters,cv=2)
# best_model.fit(x_train, y_train,early_stopping_rounds=10, eval_metric="auc",
#           eval_set=eval_set, verbose=False)
model.fit(X_train, y_train,early_stopping_rounds=10, eval_metric="auc",eval_set=eval_set, verbose=True)

In [ ]:
import sklearn

#validation
y_pred = model.predict_proba(X_test)
# y_pred = best_model.predict_proba(x_test)
#y_pred會有兩欄（加起來總共100%），不知道為什麼這裡要取第二欄
y_pred = y_pred[:,1]
# fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_valid, y_pred)
# sklearn.metrics.auc(fpr,tpr)

In [ ]:
y_pred.shape

```
submission = pd.DataFrame()

submission['id'] = test_id

submission['Fraud'] = 您的預測結果

submission.to_csv('您的檔名.csv',index=False)```

In [ ]:
test.head()

In [ ]:
#predict
submission = pd.DataFrame()
submission['id'] = test.id - 1
submission['Fraud'] = y_pred
# submission.columns=['id','Fraud']
submission.to_csv("submission.csv", index=False)
submission.head()